## Week 0 HW2

1. This assignment is an individual effort.
2. Submission to be uploaded into your group repositories in the folder python_intro
3. Deadline is 20th of April 5:00 PM.
4. Please follow google's [python styleguide](https://google.github.io/styleguide/pyguide.html) for your code. Pay attention to the guidelines for naming convention, comments and main.
5. Code will be checked for plagiarism. Compelling signs of a duplicated effort will lead to a rejection of submission and will attract a 100\% grade penalty.


### 1
Please load the file provided to you by email. Use _json_ module to read it as a list. 

In [26]:
# 1 use open a connection to the file
# 2 read contents of the file
# 3 use the json module to read the string as a list
#(replace None with relevant code)
import re
import json

file_handle = open("1960_Eisenhower_Guglielmo.txt","r")
file_content = file_handle.read()
speech = json.loads(file_content)

# Example
speech[0]

u'1960'

Notice that type(speech) is list. Please take a moment to go through the python list documentation and check out the various ways to manipulate lists.  

### 2
The first element of the list is the year of speech, the second element is the president who gave it, while the third is the transcript of the same. 

1. Inspect the transcript. Note the commonly used non-alphanumerical characters. Use an appropriate method of strings to get rid of them.
2. Use an appropriate string method to split the string of the speech into a list of smaller list of words.
3. Convert all words into lower case and return the list. Use a for loop. Then use a list comprehension to do the same.  

In [31]:
#1 
stripped_text = re.sub(r'\W+', ' ', speech[2])

#2
word_list = stripped_text.split()

#3 

"""
The following ways two ways of writing the for loop are equivalent
square = []
num_list = [1, 2, 3, 4]
for num in num_list:
    square.append(num**2)
num_square = [num**2 for num in num_list]
print square, num_square
"""
# with the loop
lower_words = []
for num in word_list:
    lower_words.append(num.lower())
# with list comprehension
lower_words = []    
lower_words = [x.lower() for x in word_list]

# Example
lower_words[0:9]

[u'delivered',
 u'in',
 u'person',
 u'before',
 u'a',
 u'joint',
 u'session',
 u'mr',
 u'president']

### 3
Create a function _preprocess_ that takes as arguments _text_ and _non_alphanum_, a string of non-alphanumeric characters that you want get rid of. Perform all operations specfied in the previous question. However, converting to lowercase should be an optional argument. The data structure returned should be a list of words.

In [64]:
def preprocess(text, nonalphanum, lower = False):
    stripped_text = re.sub(nonalphanum, ' ',text)
    word_list = stripped_text.split()
    if lower == True:
        lower_words = [x.lower() for x in word_list]
        return lower_words
    return word_list

# Example: this eliminates "--", "-", "[" and "]" characters using re.sub function.
preprocess(speech[2], "[\-\-,\[,\]]", lower = True)[0:9]

[u'delivered',
 u'in',
 u'person',
 u'before',
 u'a',
 u'joint',
 u'session',
 u'mr.',
 u'president']

### 4
Create a function _word_freq_ that takes as input a word list that has been preprocessed and returns a dictionary of the word frequency. Which is the fourth most frequent word of your word list? (Provide code that computes it)

In [68]:
def wordfreq(word_list):
    word_set = set(word_list) # get the list of unique words
    # create the frequency dictionary
    freq = {}
    # iterate through the unique words
    for word in word_set:
        freq[word] = word_list.count(word)    
    return freq

freq = wordfreq(lower_words) 

# order the words by descending frequency
ordered = sorted(freq, key=lambda k: (freq[k]), reverse=True)  
# print the fourth element of the just created list
ordered[3] 

u'and'

### 5
Write a function that takes as input a word list and returns a dictionary of the frequencies of word lengths. Do not use the api collections for this assignment. But have a look at its [documentation](https://docs.python.org/2/library/collections.html). Its useful tool to have in your repository.

In [70]:
def wordlengths(word_list):
    lengths = [len(x) for x in word_list] 
    # save the unique lengths values
    lengths = set(lengths)
    freq = {}
    # iterate through each length value
    for l in lengths:
        freq[l] = sum(1 for x in word_list if len(x) == l)
    return freq

# Example
freq = wordlengths(lower_words)
freq    

{1: 163,
 2: 1055,
 3: 1001,
 4: 776,
 5: 587,
 6: 466,
 7: 527,
 8: 373,
 9: 267,
 10: 224,
 11: 132,
 12: 63,
 13: 47,
 14: 22,
 15: 5,
 16: 2,
 17: 1}

### 6
Load the file _sou_all.txt_ in ./data/pres_speech. Inspect its contents. Familiarise yourself with using regular expressions in python. You can use this [document](https://docs.python.org/2/howto/regex.html) as a starting point. Now use regular expressions to seperate the different speeches. Your function should accept the text and a regular expression as input and return a list of lists. Each element of the list should be a list with following structure:

1. year
2. president
3. List of the transcript of the speech broken down into paragraphs.

Save your result as json

In [4]:
import re
import json

file_handle = open("sou_all.txt","r+")
text = file_handle.read()

def sep_speeches(text, regex1, regex2):
    clean = re.sub(r'\_',' ', text) # clean the underscores
    p = re.compile(regex1)
    speeches = p.split(clean)   # split the speeches using string regex1
    speeches = speeches[1:] # remove the first element of the list, which was empty
    speeches = [w.replace('\n\n\n\n', '\n\n') for w in speeches]   # reduce spaces (this is also useful for temp3 - see later)
    final = []   # initialize the final list of lists
    for speech in speeches:
        temp = filter(None, re.split(regex2, speech))[1:]
        if temp != []:
            final.append(temp)
        else:    # if there is a * after the date, split in another way
            temp2 = filter(None, re.split(r'(\d\d\d\d+)[\*] oral (\w.*) \*\*\*\*\*\n\n', speech))[1:]
            if temp2 != []:
                final.append(temp2)
            else:   # if the speech is written also split in another way
                temp3 = filter(None, re.split(r'(\d\d\d\d+) written (\w.*) \*\*\*\*\*\n\n', speech))[1:]
                final.append(temp3)
    return final            
 
# call the function on our text and dump the result in json    
final = sep_speeches(text, '\*\*\*\*\*\*\*\*\*\*', '(\d\d\d\d+) oral (\w.*) \*\*\*\*\*\n\n')                
with open("dump.json", mode='w') as f:
    json.dump(final, f, indent = 3)  